In [12]:
%load_ext autoreload
%autoreload 2
%load_ext tensorboard

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Imports

In [2]:
from pathlib import Path
import shutil
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.callbacks import (
    ReduceLROnPlateau,
    EarlyStopping,
    ModelCheckpoint,
    TensorBoard
)
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from utils.model import make_model, freeze_all_vgg, unfreeze_last_vgg
from utils.data import train_test_valid_split, filter_binary_labels, optimize_dataset, create_split, delete_folder

## Image files management -> train, test, valid splits

In [3]:
DATASET_SOURCE_PATH = Path(r'../data/dataset')
SPLITS_DESTINATION_PATH = Path(r'../data')

TEST_SIZE = 0.15
VALID_SIZE = 0.15

X_train, X_test, X_valid = train_test_valid_split(DATASET_SOURCE_PATH, test_size=TEST_SIZE, valid_size=VALID_SIZE)

splits = [('train', X_train), ('test', X_test), ('valid', X_valid)]

for split in splits:
    destination_path = Path(SPLITS_DESTINATION_PATH) / split[0]
    delete_folder(destination_path)
    create_split(split[1], destination_path)

--------------DELETE TRAIN SPLIT------------
--------------COPY TRAIN SPLIT------------


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 823/823 [00:00<00:00, 877.66it/s]


--------------DELETE TEST SPLIT------------
--------------COPY TEST SPLIT------------


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 177/177 [00:00<00:00, 860.19it/s]


--------------DELETE VALID SPLIT------------
--------------COPY VALID SPLIT------------


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 177/177 [00:00<00:00, 916.37it/s]


## Dataset loading

In [4]:
IMG_HEIGHT = 224
IMG_WIDTH = 224
BATCH_SIZE = 64
SEED = None

train_path = SPLITS_DESTINATION_PATH / 'train'
train_ds = tf.keras.preprocessing.image_dataset_from_directory(train_path, image_size=(IMG_HEIGHT, IMG_WIDTH),\
                                                               batch_size=BATCH_SIZE, shuffle=True, \
                                                               label_mode='categorical', seed=SEED)

valid_path = SPLITS_DESTINATION_PATH / 'valid'
valid_ds = tf.keras.preprocessing.image_dataset_from_directory(valid_path, image_size=(IMG_HEIGHT, IMG_WIDTH),\
                                                               batch_size=BATCH_SIZE, shuffle=True, \
                                                               label_mode='categorical', seed=SEED)

class_names = train_ds.class_names
assert class_names == valid_ds.class_names
AUTOTUNE = tf.data.AUTOTUNE

if len(class_names) == 2:  # take the one-hot-encoded matrix of labels and convert to a vector if binary classification
    train_ds = train_ds.map(filter_binary_labels, num_parallel_calls=AUTOTUNE)
    valid_ds = valid_ds.map(filter_binary_labels, num_parallel_calls=AUTOTUNE)
train_ds = optimize_dataset(train_ds)
valid_ds = optimize_dataset(valid_ds)

Found 822 files belonging to 2 classes.
Found 177 files belonging to 2 classes.


In [ ]:
plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        label_idx = labels.numpy()[i][0] if len(class_names) == 2 else np.argmax(labels.numpy()[i], axis=0)
        plt.title(class_names[label_idx])
        plt.axis("off")

## Model

In [6]:
N_HIDDEN = 512
BASE_LR = 0.001

model = make_model(n_classes=len(class_names), n_hidden=N_HIDDEN, img_height=IMG_HEIGHT, img_width=IMG_WIDTH)
freeze_all_vgg(model)

loss = tf.keras.losses.CategoricalCrossentropy() if len(class_names) > 2 else tf.keras.losses.BinaryCrossentropy()
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=BASE_LR),
              loss=loss, metrics=['accuracy'])
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 sequential (Sequential)     (None, 224, 224, 3)       0         
                                                                 
 tf.__operators__.getitem (S  (None, 224, 224, 3)      0         
 licingOpLambda)                                                 
                                                                 
 tf.nn.bias_add (TFOpLambda)  (None, 224, 224, 3)      0         
                                                                 
 vgg16 (Functional)          (None, 512)               14714688  
                                                                 
 flatten (Flatten)           (None, 512)               0         
                                                             

#### Classifier initial training

In [7]:
# TODO - delete last trained model

In [8]:
LOG_PATH = Path(r'../models/vgg16/logs')
CHECKPOINTS_PATH = Path(r'../models/vgg16/checkpoints')
BASE_EPOCHS = 30

tb = TensorBoard(log_dir=LOG_PATH)
checkpoint = ModelCheckpoint(CHECKPOINTS_PATH / 'train_{epoch}.tf', verbose=1, save_weights_only=True,
                             save_best_only=True, monitor='val_loss')
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=4, verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=15, verbose=1)

history = model.fit(train_ds, epochs=BASE_EPOCHS, validation_data=valid_ds, callbacks=[tb, checkpoint, reduce_lr,
                                                                                       early_stopping])

Epoch 1/30
13/13 [==============================] - ETA: 0s - loss: 0.7486 - accuracy: 0.6630
Epoch 00001: val_loss improved from inf to 0.98443, saving model to ..\models\vgg16\checkpoints\train_1.tf
13/13 [==============================] - 52s 3s/step - loss: 0.7486 - accuracy: 0.6630 - val_loss: 0.9844 - val_accuracy: 0.7514 - lr: 0.0010
Epoch 2/30
13/13 [==============================] - ETA: 0s - loss: 0.5179 - accuracy: 0.7603
Epoch 00002: val_loss improved from 0.98443 to 0.57598, saving model to ..\models\vgg16\checkpoints\train_2.tf
13/13 [==============================] - 7s 553ms/step - loss: 0.5179 - accuracy: 0.7603 - val_loss: 0.5760 - val_accuracy: 0.7966 - lr: 0.0010
Epoch 3/30
13/13 [==============================] - ETA: 0s - loss: 0.4122 - accuracy: 0.8333
Epoch 00003: val_loss improved from 0.57598 to 0.53732, saving model to ..\models\vgg16\checkpoints\train_3.tf
13/13 [==============================] - 7s 552ms/step - loss: 0.4122 - accuracy: 0.8333 - val_loss: 0.

Epoch 27/30
13/13 [==============================] - ETA: 0s - loss: 0.2412 - accuracy: 0.8978
Epoch 00027: val_loss did not improve from 0.51335
13/13 [==============================] - 7s 534ms/step - loss: 0.2412 - accuracy: 0.8978 - val_loss: 0.5506 - val_accuracy: 0.8079 - lr: 1.0000e-07
Epoch 00027: early stopping


#### Fine tuning

In [9]:
FINE_TUNE_AT_LAYER = 15
FINE_TUNING_EPOCHS = 30
FINE_TUNING_LR = 0.001
FINAL_MODEL_NAME = 'trained_weights'
FINAL_MODEL_SAVE_PATH = CHECKPOINTS_PATH / FINAL_MODEL_NAME

unfreeze_last_vgg(model, which_freeze=FINE_TUNE_AT_LAYER)

total_epochs = BASE_EPOCHS + FINE_TUNING_EPOCHS
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=FINE_TUNING_LR),
              loss=loss, metrics=['accuracy'])
history = model.fit(train_ds, epochs=total_epochs, validation_data=valid_ds, callbacks=[tb, checkpoint, reduce_lr, early_stopping], \
                    initial_epoch=history.epoch[-1])

model.save_weights(FINAL_MODEL_SAVE_PATH)

Epoch 27/60
13/13 [==============================] - ETA: 0s - loss: 0.7004 - accuracy: 0.6861
Epoch 00027: val_loss did not improve from 0.51335
13/13 [==============================] - 10s 651ms/step - loss: 0.7004 - accuracy: 0.6861 - val_loss: 10.2026 - val_accuracy: 0.4124 - lr: 0.0010
Epoch 28/60
13/13 [==============================] - ETA: 0s - loss: 0.6755 - accuracy: 0.6679
Epoch 00028: val_loss did not improve from 0.51335
13/13 [==============================] - 8s 594ms/step - loss: 0.6755 - accuracy: 0.6679 - val_loss: 8.2976 - val_accuracy: 0.3842 - lr: 0.0010
Epoch 29/60
13/13 [==============================] - ETA: 0s - loss: 0.5936 - accuracy: 0.7311
Epoch 00029: val_loss did not improve from 0.51335
13/13 [==============================] - 8s 594ms/step - loss: 0.5936 - accuracy: 0.7311 - val_loss: 4.2594 - val_accuracy: 0.3898 - lr: 0.0010
Epoch 30/60
13/13 [==============================] - ETA: 0s - loss: 0.5396 - accuracy: 0.7360
Epoch 00030: val_loss improved fr

13/13 [==============================] - ETA: 0s - loss: 0.3306 - accuracy: 0.8467
Epoch 00053: val_loss did not improve from 0.40245

Epoch 00053: ReduceLROnPlateau reducing learning rate to 1.000000082740371e-08.
13/13 [==============================] - 8s 594ms/step - loss: 0.3306 - accuracy: 0.8467 - val_loss: 0.4094 - val_accuracy: 0.8305 - lr: 1.0000e-07
Epoch 54/60
13/13 [==============================] - ETA: 0s - loss: 0.3314 - accuracy: 0.8516
Epoch 00054: val_loss did not improve from 0.40245
13/13 [==============================] - 8s 593ms/step - loss: 0.3314 - accuracy: 0.8516 - val_loss: 0.4097 - val_accuracy: 0.8305 - lr: 1.0000e-08
Epoch 55/60
13/13 [==============================] - ETA: 0s - loss: 0.3547 - accuracy: 0.8394
Epoch 00055: val_loss did not improve from 0.40245
13/13 [==============================] - 8s 593ms/step - loss: 0.3547 - accuracy: 0.8394 - val_loss: 0.4098 - val_accuracy: 0.8305 - lr: 1.0000e-08
Epoch 56/60
13/13 [==============================

#### Model evaluation

In [10]:
# TODO - load model if not trained

In [58]:
test_path = Path(r'../data/test')
test_ds = tf.keras.preprocessing.image_dataset_from_directory(test_path, image_size=(IMG_HEIGHT, IMG_WIDTH), \
                                                              batch_size=BATCH_SIZE, shuffle=False, \
                                                              label_mode='categorical')
assert class_names == test_ds.class_names

if len(class_names) == 2:  # take the one-hot-encoded matrix of labels and convert to a vector if binary classification
    test_ds = test_ds.map(filter_binary_labels, num_parallel_calls=AUTOTUNE)
test_ds = optimize_dataset(test_ds)

metrics = model.evaluate(test_ds)
print('Loss: {} --------- Accuracy: {}%'.format(metrics[0], np.round(metrics[1]*100, 2)))

y_pred = model.predict(test_ds)
y_true = tf.concat([y for x, y in test_ds], axis=0)
if len(class_names) == 2: # uses a threshold for the predictions if binary classification problem
    y_pred[y_pred >= 0.5] = 1
    y_pred[y_pred < 0.5] = 0
    y_true = y_true.numpy()
else: # uses argmax if not binary classification
    y_pred = np.argmax(y_pred, axis=1)
    y_true = np.argmax(y_true.numpy(), axis=1)

print(classification_report(y_true, y_pred, target_names=class_names, digits=2))

pred_labels = [('PRED_' + class_name) for class_name in class_names]
real_labels = [('REAL_' + class_name) for class_name in class_names]
pd.DataFrame(confusion_matrix(y_true, y_pred), columns=pred_labels, index=real_labels)

Found 177 files belonging to 2 classes.
3/3 [==============================] - 2s 387ms/step - loss: 0.4649 - accuracy: 0.8079
Loss: 0.4649169445037842 --------- Accuracy: 80.79%
              precision    recall  f1-score   support

           0       0.85      0.80      0.82       100
           1       0.76      0.82      0.79        77

    accuracy                           0.81       177
   macro avg       0.81      0.81      0.81       177
weighted avg       0.81      0.81      0.81       177



,PRED_0,PRED_1
REAL_0,80,20
REAL_1,14,63
